In [1]:
import torch
import pandas as pd

/storage/miniconda3/envs/nbfnet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# test_output = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/test_output.pt')
# test_output = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/experiments/NBFNet/FB15k-237/2024-09-10-05-23-42/test_output.pt')
id2relation = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/id2relation-fb15k237.pt')
# id2relation = torch.load('/storage/ryoji/Graph-Transformer/NBFNet-PyG/id2relation-wn18rr.pt')

In [4]:
test_df = pd.DataFrame(test_output)

In [9]:
test_df

,Ranking,Rel,Mode
0,1,/education/educational_institution/colors,tail
1,1,/film/film/costume_design_by,tail
2,27,/celebrities/celebrity/celebrity_friends./cele...,tail
3,1,/education/educational_institution/colors,tail
4,1,/education/educational_institution/colors,tail
...,...,...,...
6265,1,/education/educational_institution/colors_inv,head
6266,1,/award/award_category/winners./award/award_hon...,head
6267,1412,/film/film/costume_design_by_inv,head
6268,1,/award/award_category/winners./award/award_hon...,head


In [5]:
test_df['Rel'] = test_df['Rel'].map(id2relation)
test_df['Mode'] = test_df['Mode'].replace({0: 'tail', 1: 'head'})

In [6]:
# Function to calculate hits@N for each mode
def calculate_hits(df, column_name, ranks):
    hits_results = {}

    # Iterate over each mode
    for m in df[column_name].unique():
        hits_results[m] = {}
        m_df = df[df[column_name] == m].sort_values(by='Ranking')

        # Calculate hits@N for each specified rank
        for n in ranks:
            hits_at_n = m_df[m_df['Ranking'] <= n].shape[0]
            total = m_df.shape[0]
            hits_results[m][f'hits@{n}'] = hits_at_n / total

    return pd.DataFrame(hits_results)
# Specify the ranks you want to calculate hits for
ranks = [1, 3, 10, 50, 100]
# Calculate hits for each mode
hits_df = calculate_hits(test_df, 'Mode', ranks)
display(hits_df)

,tail,head
hits@1,0.504625,0.487081
hits@3,0.581180,0.572568
hits@10,0.673046,0.648485
hits@50,0.772568,0.751515
hits@100,0.800000,0.778309


In [7]:
test_df.loc[test_df['Mode'] == 'head', 'Rel'] = test_df['Rel'] + '_inv'

In [8]:
hits_df = calculate_hits(test_df, 'Rel', ranks)
display(hits_df.T.sort_index(axis=0))

,hits@1,hits@3,hits@10,hits@50,hits@100
/award/award_category/winners./award/award_honor/award_winner,0.974359,0.974359,0.974359,0.974359,0.974359
/award/award_category/winners./award/award_honor/award_winner_inv,0.974359,0.974359,0.974359,0.974359,0.974359
/award/award_nominee/award_nominations./award/award_nomination/nominated_for,0.578947,0.666667,0.763158,0.877193,0.929825
/award/award_nominee/award_nominations./award/award_nomination/nominated_for_inv,0.105263,0.236842,0.350877,0.517544,0.587719
/celebrities/celebrity/celebrity_friends./celebrities/friendship/friend,0.324111,0.490119,0.735178,0.885375,0.932806
/celebrities/celebrity/celebrity_friends./celebrities/friendship/friend_inv,0.308300,0.494071,0.624506,0.798419,0.845850
/education/educational_institution/colors,0.953488,0.964651,0.971163,0.983256,0.983256
/education/educational_institution/colors_inv,0.954419,0.963721,0.973953,0.980465,0.984186
/film/film/costume_design_by,0.207034,0.302958,0.424460,0.584333,0.620304
/film/film/costume_design_by_inv,0.203038,0.325340,0.426059,0.560352,0.592326


In [10]:
hits_df.T.sort_index(axis=0).to_csv('fb15k237_per_rel_performance.csv')

In [27]:
test_output = pd.read_csv('fb15k237_per_rel_performance.csv')

In [8]:
wandb.login()

NameError: name 'wandb' is not defined

In [9]:
import wandb

In [10]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: oge208. Use `wandb login --relogin` to force relogin


True

In [63]:
run = wandb.init(entity="team_ryoji",project="KG-FB15K-237", name = "NBFNet")

In [64]:
result = {}

for index, row in test_output.iterrows():
    for i, (column, value) in enumerate(row.items()):
        if i == 0:
            if value.endswith("_inv"):
                rel = value[:-4]+"_head"
            else:
                rel = value+"_tail"
            continue
                
        result[f'test/{column}_{rel}'] = value*100

In [65]:
result

{'test/hits@1_/award/award_category/category_of_tail': 100.0,
 'test/hits@3_/award/award_category/category_of_tail': 100.0,
 'test/hits@10_/award/award_category/category_of_tail': 100.0,
 'test/hits@50_/award/award_category/category_of_tail': 100.0,
 'test/hits@100_/award/award_category/category_of_tail': 100.0,
 'test/hits@1_/award/award_category/category_of_head': 100.0,
 'test/hits@3_/award/award_category/category_of_head': 100.0,
 'test/hits@10_/award/award_category/category_of_head': 100.0,
 'test/hits@50_/award/award_category/category_of_head': 100.0,
 'test/hits@100_/award/award_category/category_of_head': 100.0,
 'test/hits@1_/award/award_category/disciplines_or_subjects_tail': 73.33333333333333,
 'test/hits@3_/award/award_category/disciplines_or_subjects_tail': 93.33333333333333,
 'test/hits@10_/award/award_category/disciplines_or_subjects_tail': 93.33333333333333,
 'test/hits@50_/award/award_category/disciplines_or_subjects_tail': 93.33333333333333,
 'test/hits@100_/award/awa

In [66]:
run.log(result, commit=True)

In [67]:
run.finish()

test/hits@100_/award/award_category/category_of_head,▁
test/hits@100_/award/award_category/category_of_tail,▁
test/hits@100_/award/award_category/disciplines_or_subjects_head,▁
test/hits@100_/award/award_category/disciplines_or_subjects_tail,▁
test/hits@100_/award/award_category/nominees./award/award_nomination/nominated_for_head,▁
test/hits@100_/award/award_category/nominees./award/award_nomination/nominated_for_tail,▁
test/hits@100_/award/award_category/winners./award/award_honor/award_winner_head,▁
test/hits@100_/award/award_category/winners./award/award_honor/award_winner_tail,▁
test/hits@100_/award/award_category/winners./award/award_honor/ceremony_head,▁
test/hits@100_/award/award_category/winners./award/award_honor/ceremony_tail,▁
test/hits@100_/award/award_ceremony/awards_presented./award/award_honor/award_winner_head,▁
